# Importation des Package

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib
import bs4
import re
import copy
import collections as co
import plotly
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np


# Webscrapping du top 10 netflix journalier

#### Fonction qui récupère le dataframe quand on lui donne la date sous forme string "aaaa/mm/jj", url est un argument conditionnel.

In [3]:
def recup_top_netflix_gen(date,url="https://www.the-numbers.com/home-market/netflix-daily-chart/"):
    
    url_enrichi = url+date
    from urllib.request import Request, urlopen
    req_text = urlopen(Request(url_enrichi, headers={'User-Agent': 'Mozilla/5.0'})).read()
    page = bs4.BeautifulSoup(req_text, "lxml")
    #print (page)

    """On récupère les données------------------"""
    #tableau=page.find("table",{'class' : 'chart full-width'})
    tableau=page.find("div",{'id':'page_filling_chart'})
    #print(tableau)
    #table_body=tableau.find('tbody')
    #print(table_body)
    rows = tableau.find_all('tr')
    #print (rows)

  
    """On récupére tous les élements correspondant au titre et à la note et On crée un dictionnaire dans lequel on met le contenu des lignes nettoyées
    L'index est le titre, la valeur la note.
    --------------"""
    new_rows=[]#on crée une liste vide dans laquelle on va mettre le contenu des lignes
    new_cols = 10*[[]]
    dico_top_netflix = dict()
    i = 0
    for row in rows:
        cols = row.find_all('td') # c'est une liste
        cols = [ele.text.strip() for ele in cols]
        #print (cols)
        if len(cols)> 0:
            dico_top_netflix[cols[0]] = cols[1:]
    #print(dico_top_netflix)    

    """"""

    """On crée ENFIN le dataframe"""
    df_top_netflix_gen = pd.DataFrame.from_dict(dico_top_netflix,orient='index')

    """On rajoute une index différente du titre"""
    for row in rows:
        cols = row.find_all('th')
        #print(cols)
        if len(cols) > 0 : 
            cols = [ele.get_text(separator=' ').strip().title() for ele in cols]
            columns_title = cols
    #print (columns_title)
    

    """On change le nom des colonnes"""
    df_top_netflix_gen.columns=columns_title[1:]
    
    """On vire les colonnes qui ne nous intéresse pas"""
    del(df_top_netflix_gen['Yd'])
    del(df_top_netflix_gen['Lw'])
    del(df_top_netflix_gen['Watch Now'])
    
    """On rajoute une colonne 'Measure Date' """
    df_top_netflix_gen['Measure Date']= [date]*df_top_netflix_gen.shape[0]
    #print ("Graph au "+ date)
    return (df_top_netflix_gen)

## Concaténer les tableaux

Liste de toutes les dates valides

In [4]:
liste_dates_valides=[]
for m in range (3,13):
    m=str(m)
    if len(m)==1:
        m = "0"+m
    for d in range (1,32):
        d=str(d)
        if len(d)==1:
            d="0"+d
        date = "2020/"+m+"/"+d
        liste_dates_valides.append(date)

#On supprime les dates sans valeurs
for i in ['2020/03/01', '2020/03/02', '2020/03/03', '2020/03/04', '2020/03/05', '2020/03/06', '2020/03/07', '2020/03/08', 
          '2020/03/09', '2020/03/10', '2020/03/11', '2020/03/12', '2020/03/13', '2020/03/14', '2020/03/15', '2020/03/16', 
          '2020/03/17', '2020/03/18', '2020/03/19', '2020/03/20', '2020/03/21', '2020/03/22', '2020/03/23', '2020/03/28', 
          '2020/03/29', '2020/04/04', '2020/04/05', '2020/04/11', '2020/04/12', '2020/04/18', '2020/04/19', '2020/04/25', 
          '2020/04/26', '2020/08/09', '2020/08/30', '2020/12/10', '2020/12/11', '2020/12/12', '2020/12/13', '2020/12/14',
          '2020/12/15', '2020/12/16', '2020/12/17', '2020/12/18', '2020/12/19', '2020/12/20', '2020/12/21', '2020/12/22', 
          '2020/12/23', '2020/12/24', '2020/12/25', '2020/12/26', '2020/12/27', '2020/12/28', '2020/12/29', '2020/12/30', 
          '2020/12/31', '2020/04/31', '2020/06/31', '2020/09/31', '2020/11/31', '2020/08/06']:
    liste_dates_valides.remove(i)


In [5]:
df_06_05=recup_top_netflix_gen("2020/06/05")
df_06_06=recup_top_netflix_gen("2020/06/06")
DF=pd.concat([df_06_05, df_06_06])
DF.sort_values('Measure Date',ascending=False)


,Title,Type,Netflix Excl.,Netflix Release Date,Days In Top 10,Viewer- Ship Score,Measure Date
8,Clueless,Movie,,,4,11,2020/06/06
7,The Help,Movie,,,4,17,2020/06/06
6,Sweet Magnolias,TV Show,Yes,"May 19, 2020",18,142,2020/06/06
5,Avatar the Last Airbender,TV Show,,,19,145,2020/06/06
4,Cloudy with a Chance of Mea…,Movie,,,2,7,2020/06/06
3,The Last Days of American C…,Movie,Yes,"Jun 5, 2020",1,6,2020/06/06
2,Fuller House,TV Show,Yes,,4,31,2020/06/06
1,Jeffrey Epstein: Filthy Rich,TV Show,Yes,"May 27, 2020",10,91,2020/06/06
2,Jeffrey Epstein: Filthy Rich,TV Show,Yes,"May 27, 2020",9,83,2020/06/05
1,Space Force,TV Show,Yes,"May 29, 2020",7,70,2020/06/05


In [6]:
mars=[]
avril=[]
mai=[]
juin=[]
juillet=[]
aout=[]
septembre=[]
octobre=[]
novembre=[]
decembre=[]
for i in liste_dates_valides:
    if i <"2020/04/01":
        mars.append(i)
for i in liste_dates_valides:
    if i < "2020/05/01" and i > "2020/03/31":
        avril.append(i)
for i in liste_dates_valides:
    if i < "2020/06/01" and i > "2020/04/31":
        mai.append(i)
for i in liste_dates_valides:
    if i < "2020/07/01" and i > "2020/05/31":
        juin.append(i)
for i in liste_dates_valides:
    if i < "2020/08/01" and i > "2020/06/31":
        juillet.append(i)
for i in liste_dates_valides:
    if i < "2020/09/01" and i > "2020/07/31":
        aout.append(i)
for i in liste_dates_valides:
    if i < "2020/10/01" and i > "2020/08/31":
        septembre.append(i)
for i in liste_dates_valides:
    if i < "2020/11/01" and i > "2020/09/31":
        octobre.append(i)
for i in liste_dates_valides:
    if i < "2020/12/01" and i > "2020/10/31":
        novembre.append(i)
for i in liste_dates_valides:
    if i > "2020/11/31":
        decembre.append(i)
mois = [mars,avril,mai,juin,juillet,aout,septembre,octobre,novembre,decembre]
mois

[['2020/03/24',
  '2020/03/25',
  '2020/03/26',
  '2020/03/27',
  '2020/03/30',
  '2020/03/31'],
 ['2020/04/01',
  '2020/04/02',
  '2020/04/03',
  '2020/04/06',
  '2020/04/07',
  '2020/04/08',
  '2020/04/09',
  '2020/04/10',
  '2020/04/13',
  '2020/04/14',
  '2020/04/15',
  '2020/04/16',
  '2020/04/17',
  '2020/04/20',
  '2020/04/21',
  '2020/04/22',
  '2020/04/23',
  '2020/04/24',
  '2020/04/27',
  '2020/04/28',
  '2020/04/29',
  '2020/04/30'],
 ['2020/05/01',
  '2020/05/02',
  '2020/05/03',
  '2020/05/04',
  '2020/05/05',
  '2020/05/06',
  '2020/05/07',
  '2020/05/08',
  '2020/05/09',
  '2020/05/10',
  '2020/05/11',
  '2020/05/12',
  '2020/05/13',
  '2020/05/14',
  '2020/05/15',
  '2020/05/16',
  '2020/05/17',
  '2020/05/18',
  '2020/05/19',
  '2020/05/20',
  '2020/05/21',
  '2020/05/22',
  '2020/05/23',
  '2020/05/24',
  '2020/05/25',
  '2020/05/26',
  '2020/05/27',
  '2020/05/28',
  '2020/05/29',
  '2020/05/30',
  '2020/05/31'],
 ['2020/06/01',
  '2020/06/02',
  '2020/06/03',
  '20

In [7]:
df_mars = recup_top_netflix_gen("2020/03/24")
for i in mars[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_mars = pd.concat([df_mars,a_conc])

In [8]:
df_avril = recup_top_netflix_gen("2020/04/01")
a_conc=0
for i in avril[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_avril = pd.concat([df_avril,a_conc])

In [12]:
df_mai = recup_top_netflix_gen("2020/05/01")
a_conc=0
for i in mai[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_mai = pd.concat([df_mai,a_conc])

In [15]:
df_juin

,Title,Type,Netflix Excl.,Netflix Release Date,Days In Top 10,Viewer- Ship Score,Measure Date
1,Space Force,TV Show,Yes,"May 29, 2020",3,30,2020/06/01
2,Jeffrey Epstein: Filthy Rich,TV Show,Yes,"May 27, 2020",5,47,2020/06/01
3,Sweet Magnolias,TV Show,Yes,"May 19, 2020",13,110,2020/06/01
4,Uncut Gems,Movie,,,7,59,2020/06/01
5,Avatar the Last Airbender,TV Show,,,14,117,2020/06/01
...,...,...,...,...,...,...,...
6,Athlete A,Movie,Yes,"Jun 24, 2020",5,28,2020/06/30
7,The Nut Job,Movie,,"Aug 13, 2018",10,55,2020/06/30
8,Feel the Beat,Movie,Yes,"Jun 19, 2020",11,54,2020/06/30
9,Home Game,TV Show,Yes,"Jun 26, 2020",3,8,2020/06/30


In [13]:
df_juin = recup_top_netflix_gen("2020/06/01")
a_conc=0
for i in juin[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_juin = pd.concat([df_juin,a_conc])

In [16]:
df_juillet = recup_top_netflix_gen("2020/07/01")
a_conc=0
for i in juillet[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_juillet = pd.concat([df_juillet,a_conc])

In [17]:
df_aout = recup_top_netflix_gen("2020/08/01")
a_conc=0
for i in aout[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_aout = pd.concat([df_aout,a_conc])

In [24]:
df_octobre

,Title,Type,Netflix Excl.,Netflix Release Date,Days In Top 10,Viewer- Ship Score,Measure Date
1,American Murder: The Family…,Movie,Yes,"Sep 30, 2020",1,10,2020/10/01
2,Ratched,TV Show,Yes,"Sep 18, 2020",13,127,2020/10/01
3,Bad Teacher,Movie,,"May 27, 2013",4,29,2020/10/01
4,Enola Holmes,Movie,Yes,"Sep 30, 2020",8,71,2020/10/01
5,The Good Place,TV Show,,,5,29,2020/10/01
...,...,...,...,...,...,...,...
6,Secrets of the Saqqara Tomb,Movie,Yes,"Oct 28, 2020",2,10,2020/10/31
7,Barbarians,TV Show,Yes,"Oct 23, 2020",8,55,2020/10/31
8,Rogue City,Movie,Yes,"Oct 30, 2020",1,3,2020/10/31
9,The Haunting of Bly Manor,TV Show,Yes,"Oct 9, 2020",22,174,2020/10/31


In [18]:
df_septembre = recup_top_netflix_gen("2020/09/01")
a_conc=0
for i in septembre[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_septembre = pd.concat([df_septembre,a_conc])

In [22]:
df_octobre = recup_top_netflix_gen("2020/10/01")
a_conc=0
for i in octobre[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_octobre = pd.concat([df_octobre,a_conc])

In [25]:
df_novembre = recup_top_netflix_gen("2020/11/01")
a_conc=0
for i in novembre[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_novembre = pd.concat([df_novembre,a_conc])

In [26]:
df_decembre = recup_top_netflix_gen("2020/11/01")
a_conc=0
for i in decembre[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_decembre = pd.concat([df_decembre,a_conc])

In [27]:
def sortby(df):
    return df.sort_values(['Title','Measure Date'],ascending=(True,False))
#sortby(df_mars)

,Title,Type,Netflix Excl.,Netflix Release Date,Days In Top 10,Viewer- Ship Score,Measure Date
10,100 Humans,TV Show,Yes,"Mar 13, 2020",4,6,2020/03/27
9,100 Humans,TV Show,Yes,"Mar 13, 2020",3,5,2020/03/26
10,100 Humans,TV Show,Yes,"Mar 13, 2020",2,3,2020/03/25
9,100 Humans,TV Show,Yes,"Mar 13, 2020",1,2,2020/03/24
3,All American,TV Show,,,6,52,2020/03/31
3,All American,TV Show,,,5,44,2020/03/30
2,All American,TV Show,,,4,36,2020/03/27
2,All American,TV Show,,,3,27,2020/03/26
2,All American,TV Show,,,2,18,2020/03/25
2,All American,TV Show,,,1,9,2020/03/24


#### Création de dico_score

In [42]:
liste_df = [df_decembre,df_novembre,df_octobre,df_septembre,df_aout,df_juillet,df_juin,df_mai,df_avril,df_mars]
dico_score=dict()

for df_moi in liste_df:
    for i in range(sortby(df_moi).shape[0]):
        if sortby(df_moi)['Title'][i] not in dico_score:
            dico_score[sortby(df_moi)['Title'][i]]=sortby(df_moi)['Viewer- Ship Score'][i]

dico_score


{'Alien Worlds': '17',
 'Ava': '12',
 'Barbarians': '56',
 'Big Mouth': '45',
 'Blood of Zeus': '24',
 'Cocomelon': '167',
 'Dr. Seussâ\x80\x99 The Grinch': '96',
 'Hillbilly Elegy': '56',
 'His House': '8',
 'Holidate': '40',
 'How the Grinch Stole Christmas': '22',
 'Hubie Halloween': '140',
 'La belva': '17',
 'Marauders': '32',
 'Over the Moon': '64',
 'Peppermint': '48',
 'Rogue City': '11',
 'Rust Creek': '9',
 'Secrets of the Saqqara Tomb': '15',
 'Selena: The Series': '50',
 'The 2nd': '25',
 'The Christmas Chronicles 2': '88',
 'The Crown': '158',
 'The Haunting of Bly Manor': '177',
 "The Queen's Gambit": '381',
 'Triple 9': '6',
 'Virgin River': '108',
 '60 Days In': '4',
 'American Horror Story': '17',
 "Chappelle's Show": '57',
 'Christmas on the Square': '3',
 'Cloudy with a Chance of Mea…': '58',
 'Dash & Lily': '39',
 'Easy A': '21',
 'Grand Army': '38',
 'Hard Kill': '31',
 'Hometown Holiday': '4',
 'If Anything Happens I Love You': '4',
 'Jingle Jangle: A Christmas …'

Convertit le score en intégral

In [56]:
for cle in dico_score.keys():
    u=dico_score[cle]
    dico_score[cle] = int(u)
dico_score

{'Alien Worlds': 17,
 'Ava': 12,
 'Barbarians': 56,
 'Big Mouth': 45,
 'Blood of Zeus': 24,
 'Cocomelon': 167,
 'Dr. Seussâ\x80\x99 The Grinch': 96,
 'Hillbilly Elegy': 56,
 'His House': 8,
 'Holidate': 40,
 'How the Grinch Stole Christmas': 22,
 'Hubie Halloween': 140,
 'La belva': 17,
 'Marauders': 32,
 'Over the Moon': 64,
 'Peppermint': 48,
 'Rogue City': 11,
 'Rust Creek': 9,
 'Secrets of the Saqqara Tomb': 15,
 'Selena: The Series': 50,
 'The 2nd': 25,
 'The Christmas Chronicles 2': 88,
 'The Crown': 158,
 'The Haunting of Bly Manor': 177,
 "The Queen's Gambit": 381,
 'Triple 9': 6,
 'Virgin River': 108,
 '60 Days In': 4,
 'American Horror Story': 17,
 "Chappelle's Show": 57,
 'Christmas on the Square': 3,
 'Cloudy with a Chance of Mea…': 58,
 'Dash & Lily': 39,
 'Easy A': 21,
 'Grand Army': 38,
 'Hard Kill': 31,
 'Hometown Holiday': 4,
 'If Anything Happens I Love You': 4,
 'Jingle Jangle: A Christmas …': 34,
 'Kevin Hart: Zero F**ks Given': 73,
 'Knock Knock': 58,
 'La vita dav

### Fonction qui convertit un dico en df

In [57]:
def dict_to_df(dico,columns_titles,sortby='Score'):
    df = pd.DataFrame.from_dict(dico,orient='index')
    df.columns=columns_titles
    return df.sort_values(sortby,ascending=False)

In [66]:
df_score = dict_to_df(dico_score,['Score'])

#### Le dico pour aussi avoir le type

In [63]:
liste_df = [df_decembre,df_novembre,df_octobre,df_septembre,df_aout,df_juillet,df_juin,df_mai,df_avril,df_mars]
dico_type_score=dict()

for df_moi in liste_df:
    for i in range(sortby(df_moi).shape[0]):
        if sortby(df_moi)['Title'][i] not in dico_type_score:
            dico_type_score[sortby(df_moi)['Title'][i]]=[sortby(df_moi)['Type'][i],sortby(df_moi)['Viewer- Ship Score'][i]]

#dico_type_score

In [64]:
for cle in dico_type_score.keys():
    u=dico_type_score[cle][1]
    dico_type_score[cle][1] = int(u)
#dico_type_score


In [67]:
df_type_score=dict_to_df(dico_type_score,['Type','Score'])
df_type_score

,Type,Score
The Queen's Gambit,TV Show,381
Outer Banks,TV Show,259
Unsolved Mysteries,TV Show,242
The Umbrella Academy,TV Show,235
Cobra Kai,TV Show,217
...,...,...
How to Train Your Dragon 2,Movie,1
On My Block,TV Show,1
The Office,TV Show,1
House at the End of the Street,Movie,1


Combien on a de séries ?

In [73]:
df_type_score[['Type']].isin(['Movie']).sum() #145
df_type_score[['Type']].isin(['TV Show']).sum() #141
df_type_score[['Type']].isna().sum() #0 -> manque 3 lignes ?

Type    0
dtype: int64